In [ ]:
import pandas as pd
import numpy as np

In [2]:
#read in the shelter census data
shelter_census_df = pd.read_csv('/Users/Marcy_Student/Desktop/Food Insecurity Analysis/datasets/messy/Individual_Census_by_Borough,_Community_District,_and_Facility_Type_20260205.csv')


In [3]:
shelter_census_df.head()

,Report Date,Borough,Community Districts,Census Type,Adult Family Commercial Hotel,Adult Family Shelter,Adult Shelter,Adult Shelter Commercial Hotel,Family Cluster,Family with Children Commercial Hotel,Family with Children Shelter
0,01/31/2026,Queens,8.0,Individuals,0.0,0.0,172,0,0,158,297
1,01/31/2026,Queens,7.0,Individuals,0.0,0.0,322,0,0,76,0
2,01/31/2026,Queens,6.0,Individuals,0.0,0.0,100,0,0,0,244
3,01/31/2026,Bronx,1.0,Individuals,0.0,0.0,660,159,0,223,"1,463"
4,01/31/2026,Bronx,10.0,Individuals,0.0,0.0,0,0,0,0,529


In [4]:
# print the shape, columns, and data types of the shelter census dataset
print("\n\n--- Individual Census Dataset ---")
print(f"  4. Shelter Census by CD: {shelter_census_df.shape[0]} rows, {shelter_census_df.shape[1]} columns")
print(f"Columns: {list(shelter_census_df.columns)}")
print(f"\nData types:\n{shelter_census_df.dtypes}")



--- Individual Census Dataset ---
  4. Shelter Census by CD: 5204 rows, 11 columns
Columns: ['Report Date', 'Borough', 'Community Districts', 'Census Type', 'Adult Family Commercial Hotel', 'Adult Family Shelter', 'Adult Shelter', 'Adult Shelter Commercial Hotel', 'Family Cluster', 'Family with Children Commercial Hotel', 'Family with Children Shelter']

Data types:
Report Date                               object
Borough                                   object
Community Districts                      float64
Census Type                               object
Adult Family Commercial Hotel            float64
Adult Family Shelter                     float64
Adult Shelter                             object
Adult Shelter Commercial Hotel            object
Family Cluster                            object
Family with Children Commercial Hotel     object
Family with Children Shelter              object
dtype: object


In [5]:
print(f"\nMissing values:\n{shelter_census_df.isna().sum()}\n")
print(f"Duplicate rows: {shelter_census_df.duplicated().sum()}")


Missing values:
Report Date                                 0
Borough                                     0
Community Districts                        25
Census Type                                 0
Adult Family Commercial Hotel            4972
Adult Family Shelter                     3946
Adult Shelter                            1959
Adult Shelter Commercial Hotel           3176
Family Cluster                           4747
Family with Children Commercial Hotel    3010
Family with Children Shelter             1519
dtype: int64

Duplicate rows: 0


In [6]:
#rename columns for consistency
shelter_census_df.columns = shelter_census_df.columns.str.lower().str.replace(' ', '_')

In [7]:
# Convert report_date to datetime, coercing errors to NaT
shelter_census_df['report_date'] = pd.to_datetime(shelter_census_df['report_date'], format='%m/%d/%Y', errors='coerce')


In [9]:
# print more details about the shelter census dataset like date range, unique report dates, unique boroughs, and unique community districts per borough
print(f"\nDate Range: {shelter_census_df['report_date'].min()} to {shelter_census_df['report_date'].max()}")
print(f"\nUnique Report Dates: {shelter_census_df['report_date'].nunique()}")
print(f"\nUnique Boroughs: {shelter_census_df['borough'].unique()}")
print(f"\nUnique Community Districts per Borough:")
print(shelter_census_df.groupby('borough')['community_districts'].nunique())


Date Range: 2018-07-31 00:00:00 to 2026-01-31 00:00:00

Unique Report Dates: 88

Unique Boroughs: ['Queens' 'Bronx' 'Manhattan' 'Brooklyn' 'Staten Island' 'Westchester']

Unique Community Districts per Borough:
borough
Bronx            12
Brooklyn         18
Manhattan        12
Queens           14
Staten Island     3
Westchester       0
Name: community_districts, dtype: int64


- There is a big gap in the dates from 2018 to 2019. We may need to considere that gap for our ARIMAX time series forescasting model. There are 90 months from 2018-07-31 to 2026-01-31 and here we have 88 months, we missing 2. 

In [10]:
shelter_census_clean = shelter_census_df.copy()

In [11]:
# Convert numeric columns to numeric types, coercing errors to NaN
numeric_cols = ['adult_family_commercial_hotel','adult_family_shelter','adult_shelter','adult_shelter_commercial_hotel', 'family_cluster', 
                'family_with_children_commercial_hotel', 'family_with_children_shelter']

for col in numeric_cols:
    # Remove commas and convert to numeric
    shelter_census_clean[col] = pd.to_numeric(
        shelter_census_clean[col].astype(str).str.replace(',', ''), 
        errors='coerce'
    )

In [12]:
# Extract year and month
shelter_census_clean['Year'] = shelter_census_clean['report_date'].dt.year
shelter_census_clean['Month'] = shelter_census_clean['report_date'].dt.month
shelter_census_clean['YearMonth'] = shelter_census_clean['report_date'].dt.to_period('M')

In [13]:
shelter_census_clean.head()

,report_date,borough,community_districts,census_type,adult_family_commercial_hotel,adult_family_shelter,adult_shelter,adult_shelter_commercial_hotel,family_cluster,family_with_children_commercial_hotel,family_with_children_shelter,Year,Month,YearMonth
0,2026-01-31,Queens,8.0,Individuals,0.0,0.0,172.0,0.0,0.0,158.0,297.0,2026,1,2026-01
1,2026-01-31,Queens,7.0,Individuals,0.0,0.0,322.0,0.0,0.0,76.0,0.0,2026,1,2026-01
2,2026-01-31,Queens,6.0,Individuals,0.0,0.0,100.0,0.0,0.0,0.0,244.0,2026,1,2026-01
3,2026-01-31,Bronx,1.0,Individuals,0.0,0.0,660.0,159.0,0.0,223.0,1463.0,2026,1,2026-01
4,2026-01-31,Bronx,10.0,Individuals,0.0,0.0,0.0,0.0,0.0,0.0,529.0,2026,1,2026-01


In [ ]:
# Create a new column for total shelter population by summing the relevant columns
#shelter_census_clean['total_shelter_population'] = shelter_census_clean[numeric_cols].sum(axis=1)

In [14]:
shelter_census_clean.to_csv('/Users/Marcy_Student/Desktop/Food Insecurity Analysis/datasets/cleaned/shelter_census_clean.csv', index=False)